In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import pandas as pd
from collections import Counter
from scipy.stats import wilcoxon
from IPython.core.debugger import set_trace

np.random.seed(97)

In [ ]:
# contrege results
all_sets = ["contrege_comp", "contrege_incomp", "incontrege"]
num_sets = 5
sets_dir = "features"
y_dir = "ancestor_information_analysis"
clf = LogisticRegression(random_state=0, multi_class="multinomial", max_iter=5000)

y_labels = []
all_y = []
for level in range(2,10):
    y = np.load("{}/level_{}_ancestors.npy".format(y_dir, level))
    y_mod = np.argmax(y, axis = 1)
    all_y.append(y_mod)   
    y_labels.append("level_{}_ancestors".format(level))
    
results = []
for se in range(len(all_sets)):
    set_name = all_sets[se]
    print(set_name)
    all_x = []
    for i in range(num_sets):
        x = np.load("{}/{}_set_{}.npy".format(sets_dir, set_name, i))
        all_x.append(x)
        
    print(len(all_x))
    print(all_x[0].shape)
    result = {"Feature": set_name}
    
    for i in range(len(all_y)):
        agg = 0.0
        all_scores = [0.0]*10
        all_chance = []
        for x_it in all_x:
            skf = StratifiedKFold(n_splits=10)
            scores = cross_val_score(clf, x_it, all_y[i], cv=skf, scoring="accuracy")
            agg += scores.mean()
            for j in range(scores.shape[0]):
                all_scores[j] += scores[j]/len(all_x)
            
        for train_index, test_index in skf.split(all_x[0], all_y[i]):
            y_train, y_test = all_y[i][train_index], all_y[i][test_index]
            c = Counter(list(y_train))
            majority_label, count = c.most_common()[0]
            all_chance.append(float(count) / y_train.shape[0])
        
        print(all_scores)
        agg /= len(all_x)
        result[y_labels[i]] = agg
        result["p_val_" + y_labels[i]] = wilcoxon(all_scores, all_chance, zero_method="zsplit").pvalue
    print(result)
    results.append(result)

In [ ]:
# other features
all_features = ["pos_dep_tags", "node_count", "syntactic_surprisal", "word_frequency", "word_length", "all_complexity_metrics", "incremental_bert_embeddings_layer12_PCA_dims_15"]
features_dir = "features"
y_dir = "ancestor_information_analysis"
clf = LogisticRegression(random_state=0, multi_class="multinomial", max_iter=5000)

y_labels = []
all_y = []
for level in range(2,10):
    y = np.load("{}/level_{}_ancestors.npy".format(y_dir, level))
    y_mod = np.argmax(y, axis = 1)
    all_y.append(y_mod)   
    y_labels.append("level_{}_ancestors".format(level))
    
for fe in range(len(all_features)):
    feat_name = all_features[fe]
    print(feat_name)
    x = np.load("{}/{}.npy".format(features_dir, feat_name))
    x = x.reshape((x.shape[0], -1))
    
    result = {"Feature": feat_name}
    for i in range(len(all_y)):
        all_scores = []
        all_chance = []
        skf = StratifiedKFold(n_splits=10)
        scores = cross_val_score(clf, x, all_y[i], cv=skf, scoring="accuracy")
        agg = scores.mean()
        all_scores = list(scores)

        for train_index, test_index in skf.split(x, all_y[i]):
            y_train, y_test = all_y[i][train_index], all_y[i][test_index]
            c = Counter(list(y_train))
            majority_label, count = c.most_common()[0]
            all_chance.append(float(count) / y_train.shape[0])

        print(all_scores)
        result[y_labels[i]] = agg
        result["p_val_" + y_labels[i]] = wilcoxon(all_scores, all_chance, zero_method="zsplit").pvalue
        print(result["p_val_" + y_labels[i]])
    
    print(result)
#     results.append(result)

In [ ]:
np.save("syntactic_information_analysis_results.npy", results)

In [ ]:
results = np.load("syntactic_information_analysis_results.npy", allow_pickle = True).tolist()
for i in range(len(results)):
    for k in results[i]:
        if k.startswith("level"):
            results[i][k] *= 100
            results[i][k] = np.round(results[i][k], 2)

np.save("syntactic_information_analysis_results_formatted",results)

df = pd.DataFrame(results)
df.to_csv("final_syntactic_information_analysis_results.csv")

In [ ]:
# generate table for label distribution
dist = []
label_names = {0: "noun_phrase", 1: "verb_phrase", 2: "adverb_phrase", 3: "adjective_phrase", 4: "prepositional_phrase", 5: "clause", 6: "other"}

for i in range(len(all_y)):
    d = {"level" : y_labels[i]}
    for lab in range(7):
        su = (all_y[i] == lab).sum()
        d[label_names[lab]] = su
        d[label_names[lab] + "(%)"] = su/all_y[i].shape[0]
    dist.append(d)
    
dist_df = pd.DataFrame(dist)
dist_df